In [19]:
from idr import connection
import time
import multiprocessing
from itertools import repeat
import os
import shutil
import pandas as pd

conn = connection('idr.openmicroscopy.org', 'public', 'public') 

Connected to IDR ...


In [20]:
def download_screen(screen_num, path):
    screen = conn.getObject("Screen", 1101)
    for plate in screen.listChildren():
        download_plate(plate.getId(), path)

def download_plate(plate_num, path):
    plate = conn.getObject("Plate", plate_num)
    plate_name = plate.getName()
    print("Downloading " + plate_name)
    
    #make directory for the plate
    os.makedirs(path + plate_name, exist_ok=True)
    
    #iterate through wells in plate, create well dir, save annotations/images for specific well
    well_grid = plate.getWellGrid()
    for row in range(len(well_grid)):
        for col in range(len(well_grid[row])):
            well_pos = well_grid[row][col].getWellPos()
            print("Downloading well " + well_pos)
            well_dir = path + plate_name + "/" + well_pos + "/"
            os.makedirs(well_dir)
            save_annotations(well_grid[row][col], well_dir)
            save_image(well_grid[row][col], well_dir)


def save_annotations(well, path):
    annotations_list = well.listAnnotations()
    
    #annotations are in bad format so have to recreate list before converting to pandas dataframe
    new_annotations_list = []
    for annotation in annotations_list:
        for value in annotation.getValue():
            new_annotations_list.append(value)
    
    pd.DataFrame(new_annotations_list).to_csv(path + "annotations")


def save_image(well, path):
    image = well.getImage()
    #iterate through frames and save each one
    for t in range(image.getSizeT()):
        rendered_image = image.renderImage(0, t)
        rendered_image.save(path + "/frame_" +str(t)+ ".png")

In [22]:
download_screen(1101, "idr_downloads/")


# download_plate(3451, "idr_downloads/")

# pool = multiprocessing.Pool(1)
# pool.map(save_image, range(image.getSizeT()))

KeyboardInterrupt: 